In [ ]:
import cudf
import cupy
import cuml
import math

In [ ]:
X=[]
Y=[]
for i in range(60,364,4):
    X.append(math.radians(i))
    Y.append(math.sin(math.radians(i)))
print(X)
print("---")
print(Y)

[1.0471975511965976, 1.117010721276371, 1.1868238913561442, 1.2566370614359172, 1.3264502315156905, 1.3962634015954636, 1.4660765716752369, 1.53588974175501, 1.6057029118347832, 1.6755160819145565, 1.7453292519943295, 1.8151424220741028, 1.8849555921538759, 1.9547687622336491, 2.0245819323134224, 2.0943951023931953, 2.1642082724729685, 2.234021442552742, 2.303834612632515, 2.3736477827122884, 2.443460952792061, 2.5132741228718345, 2.5830872929516078, 2.652900463031381, 2.722713633111154, 2.792526803190927, 2.8623399732707004, 2.9321531433504737, 3.001966313430247, 3.07177948351002, 3.141592653589793, 3.2114058236695664, 3.2812189937493397, 3.351032163829113, 3.420845333908886, 3.490658503988659, 3.5604716740684323, 3.6302848441482056, 3.7000980142279785, 3.7699111843077517, 3.839724354387525, 3.9095375244672983, 3.9793506945470716, 4.049163864626845, 4.118977034706618, 4.1887902047863905, 4.258603374866164, 4.328416544945937, 4.39822971502571, 4.468042885105484, 4.537856055185257, 4.60

In [ ]:
X=cupy.array(X)
print(type(X))

<class 'cupy._core.core.ndarray'>


In [ ]:
X_noisy=X+cupy.random.normal(0,0.1,X.shape)
print(X_noisy)

[1.09316447 1.04639719 1.27499396 1.20779245 1.31472647 1.56540689
 1.40495908 1.54143018 1.77396425 1.62263406 1.72628114 1.61160198
 1.86681711 1.99044168 2.00913073 2.05590374 2.20442346 2.34944195
 2.31410966 2.29750482 2.44159584 2.34469804 2.51286872 2.69996999
 2.70303687 2.93005597 2.88094542 2.9453899  2.96996265 3.13916376
 3.22278296 3.11076561 3.28145452 3.31948915 3.45129081 3.58894669
 3.58174983 3.58207848 3.78249742 3.87164384 3.87206857 3.75413313
 3.9945909  3.92904127 4.04691846 3.99823548 4.17830944 4.48237011
 4.31865932 4.31961746 4.59027658 4.82105608 4.63731073 4.85080132
 4.82574047 4.9270246  5.08468696 4.94700675 5.03818815 5.25031626
 5.147909   5.46359764 5.43897404 5.44007104 5.48813754 5.38459978
 5.90012166 5.68957039 5.87023778 5.92221376 5.81271216 5.85936889
 6.14101524 6.01883667 6.31682589 6.35658757]


In [ ]:
df=cudf.DataFrame({'x':X_noisy,'y':Y})
df.insert(name='ones',value=1.0,loc=0)

In [ ]:
df

,ones,x,y
0,1.0,1.093164,8.660254e-01
1,1.0,1.046397,8.987940e-01
2,1.0,1.274994,9.271839e-01
3,1.0,1.207792,9.510565e-01
4,1.0,1.314726,9.702957e-01
...,...,...,...
71,1.0,5.859369,-2.756374e-01
72,1.0,6.141015,-2.079117e-01
73,1.0,6.018837,-1.391731e-01
74,1.0,6.316826,-6.975647e-02


In [ ]:
X=df.iloc[:,:-1]
Y=df.iloc[:,1]
print(X)
print(Y)

    ones         x
0    1.0  1.093164
1    1.0  1.046397
2    1.0  1.274994
3    1.0  1.207792
4    1.0  1.314726
..   ...       ...
71   1.0  5.859369
72   1.0  6.141015
73   1.0  6.018837
74   1.0  6.316826
75   1.0  6.356588

[76 rows x 2 columns]
0     1.093164
1     1.046397
2     1.274994
3     1.207792
4     1.314726
        ...   
71    5.859369
72    6.141015
73    6.018837
74    6.316826
75    6.356588
Name: x, Length: 76, dtype: float64


In [ ]:
from cuml.model_selection import train_test_split
Xtrain,Xtest,Ytrain,Ytest=train_test_split(X,Y,test_size=0.3,random_state=42)

In [ ]:
l=0.01
A=Xtrain.values.T.dot(Xtrain.values)
print(A.shape)
B=l*cupy.identity(A.shape[0])
C=A+B #why cupy.add
D=cupy.linalg.inv(C)
Beta=D.dot(Xtrain.values.T.dot(Ytrain.values))
print(A)
print(B)
print(C)

(2, 2)
[[ 54.         190.68291444]
 [190.68291444 790.6506989 ]]
[[0.01 0.  ]
 [0.   0.01]]
[[ 54.01       190.68291444]
 [190.68291444 790.6606989 ]]


In [ ]:
print(Beta)

[3.00590515e-04 9.99914859e-01]


In [ ]:
Ypredict=Xtest.values.dot(Beta)
from cuml.metrics import r2_score
r2_score(Ytest,Ypredict)

0.9999999921438026

In [ ]:
from cuml.model_selection import train_test_split
XtrainN,XValid,YtrainN,YValid=train_test_split(Xtrain,Ytrain,test_size=0.3,random_state=42)

In [ ]:
lamda=[1e-7,1e-6,1e-5,1e-4,1e-3,1e-2,1e-1,1,10,100,1000]
R2_score=[]
for l in lamda:
    A=XtrainN.values.T.dot(XtrainN.values)
    B=l*cupy.identity(A.shape[0])
    C=A+B #why cupy.add
    D=cupy.linalg.inv(C)
    Beta=D.dot(XtrainN.values.T.dot(YtrainN.values))
    YpredictN=XValid.values.dot(Beta)
    R2_score.append([r2_score(YValid,YpredictN),l])
print(R2_score)

[[1.0, 1e-07], [0.9999999999999999, 1e-06], [0.9999999999999845, 1e-05], [0.999999999998445, 0.0001], [0.9999999998445523, 0.001], [0.9999999845073301, 0.01], [0.999998501319924, 0.1], [0.9998888115946885, 1], [0.9971974514253569, 10], [0.8823561050714976, 100], [-1.3443871268287366, 1000]]


In [ ]:
l=(max(R2_score)[1])
A=Xtrain.values.T.dot(Xtrain.values)
B=l*cupy.identity(A.shape[0])
C=A+B #why cupy.add
D=cupy.linalg.inv(C)
Beta=D.dot(Xtrain.values.T.dot(Ytrain.values))
Ypredict=Xtest.values.dot(Beta)
print(r2_score(Ytest,Ypredict))

1.0


In [ ]:
from cuml.linear_model import Ridge
rr=Ridge(alpha=1e-7)
print(X)
print(Y)
rr.fit(X,Y)

    ones         x
0    1.0  1.093164
1    1.0  1.046397
2    1.0  1.274994
3    1.0  1.207792
4    1.0  1.314726
..   ...       ...
71   1.0  5.859369
72   1.0  6.141015
73   1.0  6.018837
74   1.0  6.316826
75   1.0  6.356588

[76 rows x 2 columns]
0     1.093164
1     1.046397
2     1.274994
3     1.207792
4     1.314726
        ...   
71    5.859369
72    6.141015
73    6.018837
74    6.316826
75    6.356588
Name: x, Length: 76, dtype: float64


Ridge()

In [ ]:
Ypred=rr.predict(Xtest)

In [ ]:
print(r2_score(Ytest,Ypred))

1.0
